In [1]:
#!/usr/bin/env python
# coding: utf-8
import numpy as np
import torch
import pandas as pd
from torch import nn
import dgl.function as fn
import torch.nn.functional as F
import gc
import random
from tqdm import tqdm
from sklearn.metrics import auc,roc_auc_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from deepctr_torch.models.deepfm import FM,DNN
from deepctr_torch.layers  import CIN,InteractingLayer,CrossNet,CrossNetMix
from deepctr_torch.models.basemodel import *
from collections import defaultdict
from torch.optim import Optimizer
import torchtext
import os
import pickle
import warnings
from train.model.trans1 import * # 转入模型

Using backend: pytorch


In [2]:
ROOT_PATH='../data/'
PREDICT_LIST=["read_comment","like", "click_avatar", "forward",'comment','follow','favorite']
test_a=pd.read_csv(ROOT_PATH+'wedata/wechat_algo_data2/test_a.csv')
feed_info=pd.read_pickle(ROOT_PATH+'tmp/cat_feed_info.pkl')
userid2nid=pickle.load(open(ROOT_PATH+'tmp/userid2nid.pkl','rb'))
feedid2nid=pickle.load(open(ROOT_PATH+'tmp/feedid2nid.pkl','rb'))
test_a=test_a.merge(feed_info[['feedid', 'authorid', 'videoplayseconds','manual_keyword_id1','manual_tag_id1']], on='feedid', how='left')
test_a['date_']=15 # 由于 复赛数据B榜不可见 这里就不放入a榜数据使用
max_day=15
for stat_cols in tqdm([['userid'],['feedid'],['authorid'], ['userid', 'authorid'],['userid', 'manual_tag_id1'],
        ['userid','manual_keyword_id1']]):
    f = '_'.join(stat_cols)
#     tmp.to_pickle('./tmp/{}_feat_{}.pkl'.format(target_day,'_'.join(stat_cols)))
    tmp=pd.read_pickle(ROOT_PATH+'tmp/{}_feat_{}.pkl'.format(15,'_'.join(stat_cols)))
    tmp=reduce_mem(tmp)
    test_a = test_a.merge(tmp, on=stat_cols + ['date_'], how='left')
    mean_tmp=pickle.load(open(ROOT_PATH+'tmp/{}_feat_mean.pkl'.format('_'.join(stat_cols)),'rb'))
    for kk,vv in mean_tmp.items():
        test_a[kk]=test_a[kk].fillna(vv) # 填充均值
test_a=reduce_mem(test_a)
gc.collect()
feat=pickle.load(open(ROOT_PATH+'tmp/feat_list.pkl','rb'))

  0%|          | 0/6 [00:00<?, ?it/s]

7.05 Mb, 7.05 Mb (0.00 %)


 17%|█▋        | 1/6 [00:02<00:10,  2.18s/it]

3.54 Mb, 3.54 Mb (0.00 %)


 33%|███▎      | 2/6 [00:05<00:10,  2.60s/it]

...

820.76 Mb, 820.76 Mb (0.00 %)


 83%|████████▎ | 5/6 [01:33<00:21, 21.90s/it]

820.76 Mb, 820.76 Mb (0.00 %)


100%|██████████| 6/6 [02:03<00:00, 20.66s/it]


Please check the latest version manually on https://pypi.org/project/deepctr-torch/#history
1532.83 Mb, 940.79 Mb (38.62 %)


In [3]:
# test_columnsillna(-1,inplace=True)
normolizer_dict=pickle.load(open(ROOT_PATH+'tmp/normolizer_dict.pkl','rb'))
for f in tqdm(feat):
    tmp=test_a[f].values.astype('float16').clip(-1,1e8)
    tmp_max=normolizer_dict[f+'_max'] # 这里 或许我得保留均值和方差
    tmp_min=normolizer_dict[f+'_min']
    test_a[f]=((tmp-tmp_min)/tmp_max).astype('float16')

100%|██████████| 96/96 [00:07<00:00, 12.68it/s]


In [13]:
feed_emb=np.load(ROOT_PATH+'tmp/feed_emb.npy')
item_texts=pickle.load(open(ROOT_PATH+'tmp/item_texts.pkl','rb'))
feed_data=pickle.load(open(ROOT_PATH+'tmp/feed_data.pkl','rb'))
user_data=pickle.load(open(ROOT_PATH+'tmp/user_data.pkl','rb'))
graph_emb=np.concatenate([np.load(ROOT_PATH+'tmp/grap_embedding32_sg2.npy'),np.load(ROOT_PATH+'tmp/grap_embedding32_hs2.npy')],axis=1)

In [14]:
tokenize = lambda x: x.split(' ')
fields = {}
examples = []
for key, texts in item_texts.items():
    if  key in ['ocr','asr','description']:
        fields[key] = torchtext.data.Field(include_lengths=True, lower=True,tokenize=tokenize, batch_first=True, fix_length=64)
    else:
        fields[key] = torchtext.data.Field(include_lengths=True, lower=True,tokenize=tokenize, batch_first=True, fix_length=5)
for i in range(len(item_texts['ocr'])):
    example = torchtext.data.Example.fromlist(
        [item_texts[key][i] for key in item_texts.keys()],
        [(key, fields[key]) for key in item_texts.keys()])  #( [feat1,feat2], [(key1,field1),(key2,field2)] )
    examples.append(example)
textset = torchtext.data.Dataset(examples, fields)
for key, field in fields.items():
    field.build_vocab(getattr(textset, key))
for field_name, field in textset.fields.items():
    examples = [getattr(textset[i], field_name) for i in range(len(textset.examples))]

    tokens, lengths = field.process(examples)

    if not field.batch_first:
        tokens = tokens.t()
    # 给feed +上文本向量
    feed_data[field_name] = tokens

for f,d in user_data.items():
    user_data[f]=d.to(torch.device('cuda'))
    
for f,d in feed_data.items():
    feed_data[f]=d.to(torch.device('cuda'))

/home/tione/notebook/envs/tf1/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/tione/notebook/envs/tf1/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [1]:
import sys
sys.path.append('./train/')

In [4]:
# model = Model(user_data,feed_data,textset
#              ,feed_embed=feed_emb,graph_emb=graph_emb) 
model = torch.load('../data/model/trans.pth')
# model.load_state_dict(model_dict.state_dict())

In [7]:
test_pred=test_pred_func(model,test_a,userid2nid,feedid2nid,feat)
for i,f in enumerate(PREDICT_LIST):
    test_a[f]=test_pred[:,i]
sub=test_a[['userid','feedid']+PREDICT_LIST]

sub.to_csv('../../upload/model2_posemb.csv',index=False)

  0%|          | 0/1039 [00:00<?, ?it/s]/home/tione/notebook/envs/tf1/lib/python3.6/site-packages/torch/nn/functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
100%|██████████| 1039/1039 [01:11<00:00, 14.52it/s]


In [7]:
ROOT_PATH='../data/'
PREDICT_LIST=["read_comment","like", "click_avatar", "forward",'comment','follow','favorite']
test_a=pd.read_csv(ROOT_PATH+'wedata/wechat_algo_data2/test_a.csv')
userid2nid=pickle.load(open(ROOT_PATH+'tmp/userid2nid.pkl','rb'))
feedid2nid=pickle.load(open(ROOT_PATH+'tmp/feedid2nid.pkl','rb'))
test_a['date_']=15 # 由于 复赛数据B榜不可见 这里就不放入a榜数据使用
hist_seq=torch.from_numpy(np.load(ROOT_PATH+'tmp/hist_list1.npy')).long()

In [8]:

src=torch.from_numpy(test_a['userid'].apply(lambda x: userid2nid[x]).values).long()
dst=torch.from_numpy(test_a['feedid'].apply(lambda x: feedid2nid[x]).values).long()
hist_id=torch.from_numpy((test_a['date_'].values-1)*len(userid2nid)).long()+src

In [10]:
 model.eval()
batch_size=4096
leng=len(test_a)
val_pred=[]
all_aucs=[]
weights=[0.30769231, 0.23076923, 0.15384615, 0.07692308, 0.07692308,0.07692308, 0.07692308]
with torch.no_grad():
    for i in tqdm(range(0,leng//batch_size+1)):
        #         print(i*batch_size,(i+1)*batch_size)
        batch_src=src[i*batch_size:(i+1)*batch_size]
        batch_dst=dst[i*batch_size:(i+1)*batch_size]
        batch_hist=hist_seq[hist_id[i*batch_size:(i+1)*batch_size]]
        pred=model(batch_src,batch_dst,batch_hist[:,:-1],batch_hist[:,-1:].cuda())
        val_pred.append(torch.cat(pred,axis=-1).sigmoid().cpu().numpy())
    val_pred=np.concatenate(val_pred,axis=0)

  0%|          | 0/1039 [00:00<?, ?it/s]/home/tione/notebook/envs/tf1/lib/python3.6/site-packages/torch/nn/functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
100%|██████████| 1039/1039 [01:38<00:00, 10.60it/s]


In [12]:
for i,f in enumerate(PREDICT_LIST):
    test_a[f]=val_pred[:,i]
sub=test_a[['userid','feedid']+PREDICT_LIST]

sub.to_csv('../../upload/trans.csv',index=False)

In [1]:
import pandas as pd

In [2]:
sub1=pd.read_csv('../../upload/dayemb_96feat_ratings_0.6978.csv')
sub2=pd.read_csv('../../upload/trans_0.695.csv')
# sub3=pd.read_csv('../../upload/ensenmbe3_contu_more.csv')

In [3]:
ACTION_LIST = ["read_comment","like", "click_avatar", "forward",'comment','follow','favorite']#
PREDICT_LIST=["read_comment","like", "click_avatar", "forward",'comment','follow','favorite']

In [11]:
df=pd.concat([sub3,sub2],axis=1)

In [4]:
tmp=sub1[PREDICT_LIST].values*0.5+sub2[PREDICT_LIST].values*0.5#+sub3[PREDICT_LIST].values*0.4

In [5]:
sub1[PREDICT_LIST]=tmp

In [6]:
sub1.to_csv('../../upload/trans+dayemb.csv',index=False)

In [13]:
df=pd.read_pickle(ROOT_PATH+'tmp/sample_feat_df.pkl').iloc[:100]

In [5]:
test_a.fillna(-1,inplace=True)
for f in tqdm(feat):
    tmp=test_a[f].values.astype('float16').clip(-1,1e8)
    tmp_max=tmp.max() # 这里 或许我得保留均值和方差
    tmp_min=tmp.min()
    test_a[f]=((tmp-tmp_min)/tmp_max).astype('float16')
# test_a.fillna(-1,inplace=True)

 66%|██████▋   | 61/92 [00:04<00:02, 12.36it/s]/home/tione/notebook/envs/tf1/lib/python3.6/site-packages/ipykernel/__main__.py:6: RuntimeWarning: divide by zero encountered in true_divide
/home/tione/notebook/envs/tf1/lib/python3.6/site-packages/ipykernel/__main__.py:6: RuntimeWarning: invalid value encountered in true_divide
100%|██████████| 92/92 [00:06<00:00, 14.20it/s]


In [9]:
sub.head(15)

,userid,feedid,read_comment,like,click_avatar,forward,comment,follow,favorite
0,175282,50458,2.116705e-02,0.008697,0.001345,0.016754,0.000004,0.000016,1.127469e-05
1,80036,42329,3.220625e-03,0.003999,0.028065,0.000361,0.000013,0.001516,7.742712e-05
2,145791,85242,4.745018e-04,0.003250,0.007136,0.000319,0.000103,0.000083,1.777745e-04
3,28430,9425,2.364895e-04,0.012196,0.033417,0.023833,0.000216,0.002219,3.379448e-05
4,44393,11866,8.018039e-04,0.001005,0.005653,0.000692,0.000004,0.000050,2.922827e-06
5,83947,8199,5.869880e-07,0.001386,0.001197,0.000004,0.000004,0.000005,9.888038e-07
6,147551,3310,1.400830e-04,0.000712,0.001187,0.000028,0.000005,0.000008,1.515313e-05
7,128347,47645,1.845777e-02,0.010705,0.009865,0.000192,0.000119,0.000095,1.165910e-04
8,173771,109162,2.794791e-03,0.001661,0.070240,0.000006,0.000074,0.000216,2.006952e-05
9,24614,69966,4.211445e-03,0.003604,0.006716,0.000322,0.000042,0.000067,1.080539e-03


In [14]:
df['userid_bgm_song_id_12day_finish_rate']

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
     ... 
95    0.0
96    0.0
97    1.0
98    0.0
99    0.0
Name: userid_bgm_song_id_12day_finish_rate, Length: 100, dtype: float16

In [15]:
tmp=pd.read_pickle(ROOT_PATH+'tmp/{}_feat_{}.pkl'.format(15,'_'.join(['userid','bgm_song_id'])))

In [41]:
for f in feat:
    print(f,a[f])

userid_12day_like_sum 0
userid_12day_like_mean 0
userid_12day_click_avatar_sum 0
userid_12day_click_avatar_mean 0
userid_12day_forward_sum 0
userid_12day_forward_mean 0
userid_12day_comment_sum 0
userid_12day_comment_mean 0
userid_12day_follow_sum 0
userid_12day_follow_mean 0
userid_12day_favorite_sum 0
userid_12day_favorite_mean 0
feedid_12day_count 0
feedid_12day_finish_rate 0
feedid_12day_read_comment_sum 0
feedid_12day_read_comment_mean 0
feedid_12day_like_sum 0
feedid_12day_like_mean 0
feedid_12day_click_avatar_sum 0
feedid_12day_click_avatar_mean 0
feedid_12day_forward_sum 0
feedid_12day_forward_mean 0
feedid_12day_comment_sum 0
feedid_12day_comment_mean 0
feedid_12day_follow_sum 0
feedid_12day_follow_mean 0
feedid_12day_favorite_sum 0
feedid_12day_favorite_mean 0
authorid_12day_count 0
authorid_12day_finish_rate 0
authorid_12day_read_comment_sum 0
authorid_12day_read_comment_mean 0
authorid_12day_like_sum 0
authorid_12day_like_mean 0
authorid_12day_click_avatar_sum 0
authorid_12

In [4]:
import sys
os.environ["CUDA_DEVICE_ORDER"]='PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#----添加model到系统环境-----------
import sys
sys.path.append('./train/')
#------加载模型1------------------
from train.model.model1 import * # 转入模型
##  处理数据---------------------------------------------
ROOT_PATH='../data/'
PREDICT_LIST=["read_comment","like", "click_avatar", "forward",'comment','follow','favorite']
test_a=pd.read_csv('../data/wedata/wechat_algo_data2/test_a.csv')
print('----开始推断1---------')
PREDICT_LIST=["read_comment","like", "click_avatar", "forward",'comment','follow','favorite']
test_a=pd.read_csv(ROOT_PATH+'wedata/wechat_algo_data2/test_a.csv')
feed_info=pd.read_pickle(ROOT_PATH+'tmp/cat_feed_info.pkl')
userid2nid=pickle.load(open(ROOT_PATH+'tmp/userid2nid.pkl','rb'))
feedid2nid=pickle.load(open(ROOT_PATH+'tmp/feedid2nid.pkl','rb'))
test_a=test_a.merge(feed_info[['feedid', 'authorid', 'videoplayseconds','manual_keyword_id1','manual_tag_id1']], on='feedid', how='left')
test_a['date_']=15 # 由于 复赛数据B榜不可见 这里就不放入a榜数据使用
max_day=15
for stat_cols in tqdm([['userid'],['feedid'],['authorid'], ['userid', 'authorid'],['userid', 'manual_tag_id1'],
        ['userid','manual_keyword_id1']]):
    f = '_'.join(stat_cols)
#     tmp.to_pickle('./tmp/{}_feat_{}.pkl'.format(target_day,'_'.join(stat_cols)))
    tmp=pd.read_pickle(ROOT_PATH+'tmp/{}_feat_{}.pkl'.format(15,'_'.join(stat_cols)))
    tmp=reduce_mem(tmp)
    test_a = test_a.merge(tmp, on=stat_cols + ['date_'], how='left')
    mean_tmp=pickle.load(open(ROOT_PATH+'tmp/{}_feat_mean.pkl'.format('_'.join(stat_cols)),'rb'))
    for kk,vv in mean_tmp.items():
        test_a[kk]=test_a[kk].fillna(vv) # 填充均值
test_a=reduce_mem(test_a)
gc.collect()
# 加载特征list-------------------------
feat=pickle.load(open(ROOT_PATH+'tmp/feat_list.pkl','rb'))
#--------------归一化分布-----------
normolizer_dict=pickle.load(open(ROOT_PATH+'tmp/normolizer_dict.pkl','rb'))
for f in tqdm(feat):
    tmp=test_a[f].values.astype('float16').clip(-1,1e8)
    tmp_max=normolizer_dict[f+'_max'] # 这里 或许我得保留均值和方差
    tmp_min=normolizer_dict[f+'_min']
    test_a[f]=((tmp-tmp_min)/tmp_max).astype('float16')
    
src=torch.from_numpy(test_a['userid'].apply(lambda x: userid2nid[x]).values).long()
dst=torch.from_numpy(test_a['feedid'].apply(lambda x: feedid2nid[x]).values).long()
test_dense=torch.from_numpy(test_a[feat].values).float()
# ------------trans-------------------------    
hist_seq=torch.from_numpy(np.load(ROOT_PATH+'tmp/hist_list1.npy')).long()
hist_id=torch.from_numpy((test_a['date_'].values-1)*len(userid2nid)).long()+src 
hist_seq2=torch.from_numpy(np.load(ROOT_PATH+'tmp/hist_list1.npy')).long()
#-----------------trans1-------------------------------------------
from train.model.trans1 import * # 转入模型
model = torch.load(ROOT_PATH+'model/trans.pth')
test_pred=test_pred_func(model,test_a,src,dst,hist_id,hist_seq,batch_size=4096*3)
test_a[PREDICT_LIST]=test_pred
subt1=test_a[['userid','feedid']+PREDICT_LIST]
#---------------trains1-hist2----------------------
model = torch.load(ROOT_PATH+'model/trans1_hist2.pth')
test_pred=test_pred_func(model,test_a,src,dst,hist_id,hist_seq2,batch_size=4096*3)
test_a[PREDICT_LIST]=test_pred
subt3=test_a[['userid','feedid']+PREDICT_LIST]
#-------------------reansv2-----------
from train.model.trans1 import * # 转入模型
model = torch.load(ROOT_PATH+'model/transv2.pth')
test_pred=test_pred_func(model,test_a,src,dst,hist_id,hist_seq,batch_size=4096*3)
test_a[PREDICT_LIST]=test_pred
subt2=test_a[['userid','feedid']+PREDICT_LIST]
#----------modelv4----------------------------
from train.model.model4 import *
model = torch.load(ROOT_PATH+'model/model4.pth')
test_pred=test_pred_func(model,test_a,src,dst,batch_size=4096*3)
test_a[PREDICT_LIST]=test_pred
sub4=test_a[['userid','feedid']+PREDICT_LIST]

#-----------------------------------------
os.environ["CUDA_DEVICE_ORDER"]='PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from train.model.model1 import * # 转入模型
model = torch.load(ROOT_PATH+'model/model1.pth')
#----------------inference-------------------------------------------------
test_pred=test_pred_func(model,test_a,src,dst,test_dense,batch_size=4096*3)
for i,f in enumerate(PREDICT_LIST):
    test_a[f]=test_pred[:,i]
sub1=test_a[['userid','feedid']+PREDICT_LIST]
#---
model = torch.load(ROOT_PATH+'model/model1-1.pth')
test_pred=test_pred_func(model,test_a,src,dst,test_dense,batch_size=4096*3)
for i,f in enumerate(PREDICT_LIST):
    test_a[f]=test_pred[:,i]
sub1_1=test_a[['userid','feedid']+PREDICT_LIST]


 #------加载模型2------------------
from train.model.model2 import * # 转入模型
model = torch.load(ROOT_PATH+'model/model2.pth')
#----------------inference-------------------------------------------------
test_pred=test_pred_func(model,test_a,src,dst,test_dense,batch_size=4096*3)
for i,f in enumerate(PREDICT_LIST):
    test_a[f]=test_pred[:,i]
sub2=test_a[['userid','feedid']+PREDICT_LIST]

##------加载模型3------------------
from train.model.model3 import * # 转入模型
model = torch.load(ROOT_PATH+'model/model3.pth')
#----------------inference-------------------------------------------------
test_pred=test_pred_func(model,test_a,src,dst,test_dense,batch_size=4096*3)
for i,f in enumerate(PREDICT_LIST):
    test_a[f]=test_pred[:,i]
sub3=test_a[['userid','feedid']+PREDICT_LIST]
#---------------------------------



----开始推断1---------


  0%|          | 0/6 [00:00<?, ?it/s]

7.05 Mb, 7.05 Mb (0.00 %)


 17%|█▋        | 1/6 [00:02<00:10,  2.11s/it]

3.54 Mb, 3.54 Mb (0.00 %)


...

 83%|████████▎ | 5/6 [01:21<00:18, 18.99s/it]

820.76 Mb, 820.76 Mb (0.00 %)


  0%|          | 0/96 [00:00<?, ?it/s]

1532.83 Mb, 940.79 Mb (38.62 %)


  0%|          | 0/347 [00:00<?, ?it/s]/home/tione/notebook/envs/tf1/lib/python3.6/site-packages/torch/nn/functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
100%|██████████| 347/347 [01:11<00:00,  4.85it/s]


In [35]:
a=pd.concat([sub1[PREDICT_LIST],sub3[PREDICT_LIST]],axis=1).corr().values
diff=0
for i in range(7):
    diff+=a[i,i+7]
diff

6.587526789486115

In [ ]:
subt1,subt2:6.7649
subt1,subt3:6.24
subt1,sub1:6.245
sub1,subt3:6.228
sub1,sub3:6.587
sub1,sub4:6.3889
sub1,sub1_1:6.6173
sub1,sub2:6.485
sub1,subt2:6.2423
subt1,sub4:6.4165

In [36]:
#---------融合---------------------
sub=sub1.copy()
sub[PREDICT_LIST]=subt1[PREDICT_LIST]*0.2+subt2[PREDICT_LIST]*0.2+subt3[PREDICT_LIST]*0.15+\
sub1[PREDICT_LIST]*0.2+sub2[PREDICT_LIST]*0.05+sub3[PREDICT_LIST]*0.05+sub4[PREDICT_LIST]*0.1+sub1_1[PREDICT_LIST]*0.05

sub.to_csv(os.path.join(ROOT_PATH+'submission','result.csv'),index=False)

